In [1]:
import numpy as np
import pandas as pd
import random
from datetime import datetime
from scipy.integrate import quad
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import time
import mplleaflet
pd.set_option('display.max_columns',60)

KeyboardInterrupt: 

## Step 1: Define Helper Functions

In [ ]:
csv_files_2017 = [('2017' + "%.2d" + '-citibike-tripdata.csv') % i for i in range(1, 13)]
csv_files_2018 = [('2018' + "%.2d" + '-citibike-tripdata.csv') % i for i in range(1, 13)]
csv_files_2019 = [('2019' + "%.2d" + '-citibike-tripdata.csv') % i for i in range(1, 13)]
csv_files = csv_files_2017 + csv_files_2018 + csv_files_2019

In [ ]:
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

In [ ]:
def to_datetime(df):
    df1 = df.copy()
    df1['starttime'] = pd.to_datetime(df1['starttime'])
    df1['stoptime'] = pd.to_datetime(df1['stoptime'])
    df1['start_date'] = df1['starttime'].dt.date
    df1['start_time'] = df1['starttime'].dt.time
    df1['start_hour'] = df1['starttime'].dt.hour
    df1['start_min'] = df1['starttime'].dt.minute
    df1['start_year'] = df1['starttime'].dt.year
    df1['start_month'] = df1['starttime'].dt.month
    df1['start_dayofweek'] = df1['starttime'].dt.weekday   # Monday is 0, Sunday is 6
    #df1['start_dayofweek'] = df1['starttime'].dt.weekday_name  # The name of day in a week (e.g. Monday)
    df1['stop_date'] = df1['stoptime'].dt.date
    df1['stop_time'] = df1['stoptime'].dt.time
    df1['stop_hour'] = df1['stoptime'].dt.hour
    df1['stop_min'] = df1['stoptime'].dt.minute
    df1['stop_dayofweek'] = df1['stoptime'].dt.weekday
    return df1

In [ ]:
def ignore_offpeak(df):
    df1 = df.copy()
    mask = df1['start_hour'].apply(lambda x: 5 <= x <= 22)
    return df1[mask]

In [ ]:
# def lonlat2mile( lon_start,lat_start,lon_end, lat_end):
#     delta_y = (lat_end - lat_start) * 69
# #     def integrand( lat ):
# #         return np.cos( np.pi * lat / 180 ) * 69.172
# #     delta_x, xerr = quad(integrand, lat_start, lat_end )
#     delta_x = (lon_end - lon_start)*53
#     return np.abs( delta_x ) + np.abs( delta_y )
# # lonlat2mile(-73.983035,40.744449,-73.948813,40.778301)

In [ ]:
def aggregated_data(df):
    # Define distance: if same start and end location, use average speed of 7.456mph to estimate distance.
    # If different locations, calculate Manhattan distance between two stations
    df['distance']=np.where(df['start_station_ID'] == df['end_station_ID'],df['trip_duration']*7.456/3600,
                            abs(df['start_station_longitude']-df['end_station_longitude'])*53+\
                            abs(df['start_station_latitude']-df['end_station_latitude'])*69)
    # Daily average of all stations for time-series analysis
    df_daily = df.groupby('start_date').agg({'trip_duration':['count','mean'],'distance':'mean'}).reset_index()
    df_daily.columns = ['start_date','trip_per_day','daily_avg_trip_duration','daily_avg_distance']
    df_daily_merged = df.merge(df_daily, how = 'left', on = 'start_date')
    # Hourly average for each station regardeless of days 
    df_hourly = df.groupby(['start_station_ID','start_hour']).\
    agg({'trip_duration':['count','mean'],'distance':'mean'}).reset_index()
    df_hourly.columns = ['start_station_ID','start_hour','trip_per_hour','hourly_avg_trip_duration','hourly_avg_distance']
    df_hourly_merged = df_daily_merged.merge(df_hourly, how = 'left', on = ['start_station_ID','start_hour'])
    # Calculate hourly trip counts, avg trip duration, and avg trip distance per station and merge to above df
    df_hourly_eachday = df.groupby(['start_station_ID','start_date','start_hour']).\
    agg({'trip_duration':['count','mean'],'distance':'mean'}).reset_index()
    df_hourly_eachday.columns = ['start_station_ID','start_date','start_hour','trip_per_hour_eachday','hourly_avg_trip_duration_eachday','hourly_avg_distance_eachday']
    df_hourly_eachday_merged = df_hourly_merged.merge(df_hourly_eachday, how = 'left', on = ['start_station_ID','start_date','start_hour'])
    return df_hourly_eachday_merged    

In [ ]:
def merge_bikecount(df):
    # groupby start station ID, date and hour to get hourly counts of trips per start station
    checkout = df.groupby(['start_station_ID','start_date','start_hour'])['trip_duration'].count().reset_index()
    checkout.columns = ['start_station_ID','start_date','start_hour','checkout_counts']
    # groupby end station ID, date, and hour to get hourly counts of trips per end station 
    checkin = df.groupby(['end_station_ID','stop_date','stop_hour'])[['trip_duration']].count().reset_index()
    checkin.columns=['end_station_ID','stop_date','stop_hour','checkin_counts']
    # Join dataframe to get station checkin and checkout counts 
    temp = pd.merge(checkout, checkin,  how='outer', left_on=['start_station_ID','start_date','start_hour'], 
                    right_on = ['end_station_ID','stop_date','stop_hour'])
    temp['start_station_ID'] = temp['start_station_ID'].fillna(temp['end_station_ID'])
    temp['start_date'] = temp['start_date'].fillna(temp['stop_date'])
    temp['start_hour'] = temp['start_hour'].fillna(temp['stop_hour'])
    temp['checkout_counts'] = temp['checkout_counts'].fillna(0)
    temp['checkin_counts'] = temp['checkin_counts'].fillna(0)
    temp = temp.drop(['end_station_ID','stop_date','stop_hour'],axis=1)
    temp.columns=['station_ID','date','hour','checkout_counts','checkin_counts']
    temp['bike_added'] = temp['checkin_counts'] - temp['checkout_counts']
    # merge orginal dataframe to get hourly checkin/checkout information for both start and stop stations 
    df_temp_merged = pd.merge(df, temp,  how='left', left_on=['start_station_ID','start_date','start_hour'], 
         right_on = ['station_ID','date','hour']).drop(['station_ID','date','hour'],axis = 1)
    df_temp_merged = pd.merge(df_temp_merged, temp,  how='left', left_on=['end_station_ID','stop_date','stop_hour'], 
         right_on = ['station_ID','date','hour']).drop(['station_ID','date','hour'],axis = 1) 
    df_temp_merged = df_temp_merged.rename(columns={'checkout_counts_x':'start_station_checkout_counts',
                                                    'checkin_counts_x':'start_station_checkin_counts',
                                                    'bike_added_x':'start_station_bike_added', 
                                                    'checkout_counts_y':'end_station_checkout_counts',
                                                    'checkin_counts_y':'end_station_checkin_counts',
                                                    'bike_added_y':'end_station_bike_added'}) 
    return df_temp_merged
    

## Merge Datasets

In [ ]:
total_time = 0
random.seed(0)
path = '../Tripdata/'
for i, csv in enumerate(csv_files):
    start_time = time.time()
    df_temp = pd.read_csv(path+csv)
    df_temp.columns = ['trip_duration','starttime','stoptime','start_station_ID','start_station_name',
                       'start_station_latitude','start_station_longitude','end_station_ID','end_station_name',
                       'end_station_latitude','end_station_longitude','bike_ID','user_type','birth_year','gender']
    df_temp = df_temp.loc[df_temp['trip_duration']<= 24*3600] # remove trips that are longer than 1 day 
    df_temp = df_temp.loc[(df_temp['start_station_latitude']>40) & (df_temp['start_station_latitude']<41)] # remove areas that are not in NYC (equator and montreal)
    df_temp = to_datetime(df_temp)
    df_temp = ignore_offpeak(df_temp)
    df_temp = merge_bikecount(df_temp)
    df_temp = aggregated_data(df_temp)
    
    # take a 5% subset of monthly file for analysis and another 5% of the remaining dataset as test dataset for ML
    rows = len(df_temp)
    size = int(rows/20)
    selected_idx = random.sample(range(1,rows), size)
    skip_idx = list(set(df_temp.index)-set(selected_idx))
    test_idx = random.sample(skip_idx,int(len(skip_idx)/20))
    df_train = df_temp.iloc[selected_idx,:]
    df_test = df_temp.iloc[test_idx,:]
    
    # save train and test datset 
    df_train.to_csv(months[i%12] + csv[:4] + 'train.csv')
    df_test.to_csv(months[i%12] + csv[:4] + 'test.csv')
    print('Finishing data extraction from ' + csv)
    timeSpent = time.time() - start_time
    print('This iteration uses %.2f'%(timeSpent))
    total_time += timeSpent
    print(total_time)

In [ ]:
train_list = []
test_list = []
for i, csv in enumerate(csv_files):
    train_list.append(months[i%12] + csv[:4] + 'train.csv')
    test_list.append(months[i%12] + csv[:4] + 'test.csv')

In [ ]:
# Merge all months train.csv into one train dataframe
train_df = pd.DataFrame()
for i in range(len(train_list)):
    temp_df = pd.read_csv(train_list[i], index_col = 0)
    train_df = pd.concat([train_df, temp_df], axis = 0)
    print('Finished ' + str(i) + ' element')
train_df.to_csv('train.csv')

In [ ]:
#Merge all months test.csv into one test dataframe
test_df = pd.DataFrame()
for i in range(len(test_list)):
    temp_df = pd.read_csv(test_list[i], index_col = 0)
    test_df = pd.concat([test_df, temp_df], axis = 0)
    print('Finished ' + str(i) + ' element')
test_df.to_csv('test.csv')

In [ ]:
train_df = pd.read_csv('train.csv', index_col = 0)

In [ ]:
test_df = pd.read_csv('test.csv', index_col = 0)

### Merge Weather Dataset

In [ ]:
weather = pd.read_csv('weather.csv',index_col = 0).reset_index()
weather = weather.fillna(0)
weather['DATE'] = pd.to_datetime(weather['DATE'])
weather['TAVG'] = (weather['TMIN']+weather['TMAX'])/2
weather['HasPRCP'] = [1 if x !=0 else 0 for x in weather['PRCP']]
weather['HasSNOW'] = [1 if x !=0 else 0 for x in weather['SNOW']]

In [ ]:
# weather

In [ ]:
train_df['start_date'] = pd.to_datetime(train_df['start_date'])
train_df_weather = train_df.merge(weather,how='left',
                                  left_on = 'start_date', right_on = 'DATE').drop(['DATE'],axis=1)

In [ ]:
test_df['start_date'] = pd.to_datetime(test_df['start_date'])
test_df_weather = test_df.merge(weather,how='left',
                                  left_on = 'start_date', right_on = 'DATE').drop(['DATE'],axis=1)

In [ ]:
train_df_weather.to_csv('train_weather.csv')
test_df_weather.to_csv('test_weather.csv')

### Load Merged Dataset

In [ ]:
train_df_weather = pd.read_csv('train_weather.csv',index_col = 0)
test_df_weather = pd.read_csv('test_weather.csv',index_col = 0)

In [ ]:
train_df_weather['Isweekday'] = [0 if (x ==5 or x==6) else 1 for x in train_df_weather['start_dayofweek'] ]

In [ ]:
# Assuming bikers during peak hours on weekdays are commuting
rushhours = [8,9,16,17,18,19]
train_df_weather['Commute'] = np.where(((train_df_weather['Isweekday']==1) &
                                        (train_df_weather['start_hour'].isin(rushhours))),1,0)
train_df_weather[['Isweekday','start_hour','Commute']]

In [ ]:
sns.countplot(x = 'Commute',data = train_df_weather)

### Compare Weekday and Weekend Activities

In [ ]:
Weekday = train_df_weather[['Isweekday','trip_per_day','daily_avg_trip_duration','daily_avg_distance']]
Weekday.columns =  ['Isweekday','Daily Trip Count','Avg Trip Duration','Avg Distance']
Weekday = Weekday.groupby('Isweekday').agg('mean').reset_index()

sns.set(rc={'figure.figsize':(11, 4)})
cols_plot = ['Daily Trip Count','Avg Trip Duration','Avg Distance']
axes = Weekday[cols_plot].plot(kind='bar', alpha=0.5, linestyle='None', figsize=(11, 9), subplots=True,rot = 0)
axes[0].set_ylabel('Daily Trip Count')
axes[1].set_ylabel('Avg Trip Duration(s)')
axes[2].set_ylabel('Avg Distance(mile)')

### Check Rush Hours 

In [ ]:
# Group by the start_hour and see the frequency correspond to each starting hour
train_df_weather.groupby('start_hour').count()[['trip_duration']].sort_values(by = 'trip_duration', ascending = False).head(6)

In [ ]:
train_df_weather.columns

In [ ]:
# Top 50 stations during rush hours on weekdays
top50_weekdays = train_df_weather.loc[train_df_weather['Isweekday']==1].groupby(['start_station_name']).agg({'trip_duration':'count','start_station_latitude':lambda x: x.iloc[0], 'start_station_longitude':lambda x: x.iloc[0]}).\
sort_values(by = 'trip_duration', ascending = False).head(50)
top50_weekdays.head()

In [ ]:
# Top 50 stations on weekends
top50_weekdays_rush_start = train_df_weather.loc[train_df_weather['Commute']==1].groupby(['start_station_name']).agg({'trip_duration':'mean','start_station_latitude':lambda x: x.iloc[0], 'start_station_longitude':lambda x: x.iloc[0]}).\
sort_values(by = 'trip_duration', ascending = False).head(50)
top50_weekdays_rush_start.head()

In [ ]:
# Top 50 stations on weekends
weekends = train_df_weather.loc[train_df_weather['Isweekday']==0].groupby(['start_station_name']).agg({'trip_per_hour':'mean','start_station_latitude':lambda x: x.iloc[0], 'start_station_longitude':lambda x: x.iloc[0]}).\
sort_values(by = 'trip_per_hour', ascending = False).tail(50)
weekends.head()

### Map for dock stations during weekdays and weekends

In [ ]:
plt.figure(figsize=(15,10))
plt.plot(top50_weekdays['start_station_longitude'].values, top50_weekdays['start_station_latitude'].values, 'ro',alpha = 0.5,markersize=8)
plt.plot(top50_weekdays_rush_start['start_station_longitude'].values, top50_weekdays_rush_start['start_station_latitude'].values, 'bv',alpha = 0.5,markersize=8)
#plt.plot(top50_weekends['start_station_longitude'].values, top50_weekends['start_station_latitude'].values, 'bv',alpha = 0.5,markersize=8)
mplleaflet.display(tiles='cartodb_positron')

### Check number of stations in different years

In [ ]:
# Check number of stations in different years
train_df_weather.loc[train_df_weather.start_year == 2019].start_station_ID.nunique() 
train_df_weather.start_station_ID.nunique() 
# 2017 has 799, 2018 has 812, 2019 has 926, unique stations
# total of 1011 stations 
station_2017 = list(train_df_weather.loc[train_df_weather.start_year == 2017].start_station_ID.unique())
station_2018 = list(train_df_weather.loc[train_df_weather.start_year == 2018].start_station_ID.unique())
station_2019 = list(train_df_weather.loc[train_df_weather.start_year == 2019].start_station_ID.unique())
remove2018 =[item for item in station_2017 if item not in station_2018]
new2018 =[item for item in station_2018 if item not in station_2017]
remove2019 = [item for item in station_2018 if item not in station_2019]
new2019 =[item for item in station_2019 if item not in station_2018]
print('Number of stations removed in 2018: %.f' %len(remove2018))
print('Number of stations removed in 2019: %.f' %len(remove2019))
print('Number of stations added in 2018: %.f' %len(new2018))
print('Number of stations added in 2019: %.f' %len(new2019))

### Seasonal Trend 

In [ ]:
# Time Series of Trip Count
daily_trip = train_df_weather[['start_date','trip_per_day','daily_avg_trip_duration','daily_avg_distance']].sort_values(by='start_date')
daily_trip.columns = ['Date','Daily Trip Count','Avg Trip Duration','Avg Distance']
daily_trip = daily_trip.groupby('Date').agg('mean').reset_index().set_index('Date')

In [ ]:
df_temp= pd.read_csv('../Tripdata/201710-citibike-tripdata.csv')
df_temp.columns = ['trip_duration','starttime','stoptime','start_station_ID','start_station_name',
                       'start_station_latitude','start_station_longitude','end_station_ID','end_station_name',
                       'end_station_latitude','end_station_longitude','bike_ID','user_type','birth_year','gender']
df_temp = df_temp.loc[df_temp['trip_duration']<= 24*3600] # remove trips that are longer than 1 day 
df_temp = to_datetime(df_temp)
df_temp = ignore_offpeak(df_temp)
df_temp = aggregated_data(df_temp)

In [ ]:
df_temp.loc[df_temp['distance']>4].sort_values(by='distance',ascending = False)

In [ ]:
sns.set(rc={'figure.figsize':(11, 4)})
cols_plot = ['Daily Trip Count','Avg Trip Duration','Avg Distance']
axes = daily_trip[cols_plot].plot(marker='.', alpha=0.5, linestyle='None', figsize=(11, 9), subplots=True)
axes[0].set_ylabel('Daily Trip Count')
axes[1].set_ylabel('Avg Trip Duration(s)')
axes[2].set_ylabel('Avg Distance(mile)')

In [ ]:
# Trip counts by month
month_trip = train_df_weather[['start_month','trip_per_day','daily_avg_trip_duration','daily_avg_distance']].sort_values(by='start_month')
month_trip.columns=['Month','Avg Trip Counts','Avg Trip Duration','Avg Distance'] 
sns.boxplot(x="Month", y="Avg Trip Counts", data=month_trip)

In [ ]:
# Trip Duration by Month
sns.boxplot(x="Month", y="Avg Trip Duration",  data=month_trip)

In [ ]:
# Trip distance by Month
sns.boxplot(x="Month", y="Avg Distance", data=month_trip)

In [ ]:
# sns.distplot(train_df.starttime)

In [ ]:
# Non typical checkout (Consider removing them )
train_df_weather.loc[train_df_weather.distance > 10].sort_values(by = 'trip_duration', ascending = False).\
head(20)[['trip_duration', 'starttime', 'stoptime', 'user_type', 'distance']]

In [ ]:
train_df_weather.loc[(train_df_weather.distance > 10) & (train_df_weather.user_type == 'Subscriber')].shape[0]/\
train_df_weather.loc[(train_df_weather.distance > 10)].shape[0]

In [ ]:
14306/2479389

In [ ]:
train_df_weather.loc[train_df_weather.user_type == 'Subscriber'].shape[0]/train_df_weather.shape[0]

In [ ]:
train_df_weather.loc[train_df_weather.user_type != 'Subscriber'].shape[0]/train_df_weather.shape[0]

In [ ]:
a = train_df_weather.groupby('start_month').mean()[['trip_duration']].reset_index()
sns.barplot(x = a.start_month, y = a.trip_duration)

In [ ]:
b = train_df_weather.groupby('start_month').count()[['trip_duration']].reset_index()
sns.barplot(x = b.start_month, y = b.trip_duration)

In [ ]:
c = train_df_weather.groupby('start_month').mean()[['distance']].reset_index()
sns.barplot(x = c.start_month, y = c.distance)

## Weather Impact

### Temperature's impact on trip counts

In [ ]:
# Avg Temperature
TEMP = train_df_weather[['TAVG','trip_duration','trip_per_day','daily_avg_distance']]
TEMP.columns = ['Avg Temp','Avg Trip Duration(s)','Trip Count per Day','Avg Distance(mile)']
sns.scatterplot(x='Avg Temp',y='Trip Count per Day',data = TEMP).set(title = 'Trip Count per Day v.s. Average Temperature')


### Unusual Weather Condition's impact on Bike Count

In [ ]:
extreme_weather = train_df_weather[['HasPRCP','HasSNOW','Fog', 'Heavy_Fog', 'Thunder', 'Haze']].apply(pd.value_counts)
extreme_weather= extreme_weather.unstack().reset_index()
extreme_weather.columns=['Weather Condition','Yes/No','Trip Counts']
extreme_weather['Yes/No']=['Yes' if x==1 else 'No' for x in extreme_weather['Yes/No']]
plt.figure(figsize=(15,10))
plt.rcParams["axes.labelsize"] = 20
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.title("Weather Condition's Impact on Trip Counts", fontdict = {'fontsize' : 25})
sns.barplot(x='Weather Condition', y ='Trip Counts', hue ='Yes/No', data = extreme_weather)

### Correlation Heat Map

In [ ]:
from string import ascii_letters
import numpy as np
import pandas as pd
import seaborn as sns 

sns.set(style="white")
corr =train_df_weather.corr()

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=np.bool))
# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))
# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)
# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

In [ ]:
corr.to_csv('Correlation_Matrix.csv')

In [ ]:
corr.dtypes